In [2]:
import pandas
import networkx as nx
import pickle

In [3]:
def create_graph_object(num, name):
    if name == "fully":
        G = nx.complete_graph(num)
    elif name == "star":
        G = nx.star_graph(num - 1)
    elif name == "ring":
        G = nx.cycle_graph(num)
    elif name.startswith("ER_"):
        try:
            # Extract probability and seed from the graph_info
            parts = name.split("_")
            prob = float(parts[1])
            G = nx.erdos_renyi_graph(num, prob, seed=45)
        except (IndexError, ValueError):
            raise ValueError("Invalid format for ER graph type. Expected 'ER_prob'")
    else:
        raise ValueError("Not supported topology.")        
    return G

In [4]:
for num in [10,20,30]:
    for name in ["fully", "star", "ring", "ER_0.3", "ER_0.7", "ER_0.5"]:
        G = create_graph_object(num, name)
        with open(f'{num}_{name}.pk','wb') as f:
            pickle.dump(G, f)

In [5]:
for file_name in ['Abilene_edges.csv', 'GÉANT_edges.csv', 'synth50_edges.csv', 'rf1755_edges.csv', 'rf3967_edges.csv']:
    edges_df = pandas.read_csv(file_name)
    parts = file_name.split("_")
    topo = parts[0]
    G = nx.DiGraph()
    for _, row in edges_df.iterrows():
        G.add_edge(row['src'], row['dst'])
    adj_matrix = nx.adjacency_matrix(G).todense()
    print(file_name, len(adj_matrix))
    with open(f'{len(adj_matrix)}_{topo}.pk','wb') as f:
        pickle.dump(G, f)

Abilene_edges.csv 12
GÉANT_edges.csv 22
synth50_edges.csv 50
rf1755_edges.csv 87
rf3967_edges.csv 79
